# Feature Pipeline using Synthetic Data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/featurestoreorg/serverless-ml-course/blob/main/src/02-module/2_cc_feature_pipeline.ipynb)

**Note**: you may get an error when installing hopsworks on Colab, and it is safe to ignore it.

## 🗒️ This notebook is divided in 2 sections:
1. Reading the synthetic credit card data and feature engineeing,
2. Write the Pandas DataFrames to the feature groups in the feature store.


In [2]:
#!pip install -U hopsworks --quiet
!pip install -U faker --quiet

In [1]:
import pandas as pd
import datetime
import hopsworks
from sml import synthetic_data
import random
pd.options.mode.chained_assignment = None

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
start_time = (datetime.datetime.now() - datetime.timedelta(hours=24)).strftime("%Y-%m-%d %H:%M:%S")
print(start_time)

2024-03-21 12:48:30


In [3]:
#end_time = (datetime.datetime.now() - datetime.timedelta(hours=24)).strftime("%Y-%m-%d %H:%M:%S")
end_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(end_time)

2024-03-22 12:50:30


In [4]:
synthetic_data.FRAUD_RATIO = random.uniform(0.001, 0.005)
synthetic_data.TOTAL_UNIQUE_USERS = 1000
synthetic_data.TOTAL_UNIQUE_TRANSACTIONS = 54000
synthetic_data.CASH_WITHRAWAL_CARDS_TOTAL = 2000
synthetic_data.TOTAL_UNIQUE_CASH_WITHDRAWALS = 200
synthetic_data.START_DATE=start_time
synthetic_data.END_DATE=end_time

credit_cards = synthetic_data.generate_list_credit_card_numbers()
credit_cards_df = synthetic_data.create_credit_cards_as_df(credit_cards)
profiles_df = synthetic_data.create_profiles_as_df(credit_cards)
trans_df = synthetic_data.create_transactions_as_df(credit_cards)

## <span style="color:#ff5f27;"> 🛠️ Feature Engineering </span>

Fraudulent transactions can differ from regular ones in many different ways. Typical red flags would for instance be a large transaction volume/frequency in the span of a few hours. It could also be the case that elderly people in particular are targeted by fraudsters. To facilitate model learning you will create additional features based on these patterns. In particular, you will create two types of features:
1. **Features that aggregate data from different data sources**. This could for instance be the age of a customer at the time of a transaction, which combines the `birthdate` feature from `profiles.csv` with the `datetime` feature from `transactions.csv`.
2. **Features that aggregate data from multiple time steps**. An example of this could be the transaction frequency of a credit card in the span of a few hours, which is computed using a window function.

Let's start with the first category.

In [5]:
fraud_labels = trans_df.copy()[["tid", "cc_num", "datetime", "fraud_label"]]
fraud_labels

,tid,cc_num,datetime,fraud_label
0,e37c9dcafbf87c036ff582d4694ee076,4258238505394954,2024-03-21 12:48:31,0
1,9f311e76b78d7c8230042fa1dc97ef2d,4652270548978073,2024-03-21 12:48:35,0
2,ca3eb2884ce0cb3809450867e4e5777e,4797751386008277,2024-03-21 12:48:35,0
3,04d4a6b91c346c05ddf08cc910ab9670,4990402608037315,2024-03-21 12:48:36,0
4,1961400025d114658aa5dad986e90cb1,4296476972145855,2024-03-21 12:48:40,0
...,...,...,...,...
60081,38ec133a392a3448eb990e136342be0b,4336197351748828,2024-04-09 12:28:11,0
60082,1fb0747b230a5909eab063669374b843,4336197351748828,2024-04-12 14:28:11,0
60083,4edb0a8b320872288d6e3ee9a3e7d50a,4336197351748828,2024-04-15 16:28:11,0
60084,7bd96a22247346658b23f5e77199157a,4336197351748828,2024-04-18 18:28:11,0


In [6]:
from sml import cc_features

fraud_labels.datetime = fraud_labels.datetime.map(lambda x: cc_features.date_to_timestamp(x))
fraud_labels

,tid,cc_num,datetime,fraud_label
0,e37c9dcafbf87c036ff582d4694ee076,4258238505394954,1711025311000,0
1,9f311e76b78d7c8230042fa1dc97ef2d,4652270548978073,1711025315000,0
2,ca3eb2884ce0cb3809450867e4e5777e,4797751386008277,1711025315000,0
3,04d4a6b91c346c05ddf08cc910ab9670,4990402608037315,1711025316000,0
4,1961400025d114658aa5dad986e90cb1,4296476972145855,1711025320000,0
...,...,...,...,...
60081,38ec133a392a3448eb990e136342be0b,4336197351748828,1712665691000,0
60082,1fb0747b230a5909eab063669374b843,4336197351748828,1712932091000,0
60083,4edb0a8b320872288d6e3ee9a3e7d50a,4336197351748828,1713198491000,0
60084,7bd96a22247346658b23f5e77199157a,4336197351748828,1713464891000,0


In [7]:
trans_df

,tid,datetime,cc_num,category,amount,latitude,longitude,city,country,fraud_label
0,e37c9dcafbf87c036ff582d4694ee076,2024-03-21 12:48:31,4258238505394954,Electronics,456.97,43.610620,-72.972610,Rutland,US,0
1,9f311e76b78d7c8230042fa1dc97ef2d,2024-03-21 12:48:35,4652270548978073,Grocery,71.93,28.900540,-81.263670,Deltona,US,0
2,ca3eb2884ce0cb3809450867e4e5777e,2024-03-21 12:48:35,4797751386008277,Restaurant/Cafeteria,10.87,39.091120,-94.415510,Independence,US,0
3,04d4a6b91c346c05ddf08cc910ab9670,2024-03-21 12:48:36,4990402608037315,Grocery,79.64,39.717340,-74.969330,Sicklerville,US,0
4,1961400025d114658aa5dad986e90cb1,2024-03-21 12:48:40,4296476972145855,Holliday/Travel,43.42,39.083670,-84.508550,Covington,US,0
...,...,...,...,...,...,...,...,...,...,...
60081,38ec133a392a3448eb990e136342be0b,2024-04-09 12:28:11,4336197351748828,Cash Withdrawal,62.36,34.951040,-120.410433,Santa Maria,US,0
60082,1fb0747b230a5909eab063669374b843,2024-04-12 14:28:11,4336197351748828,Cash Withdrawal,79.94,34.951507,-120.407653,Santa Maria,US,0
60083,4edb0a8b320872288d6e3ee9a3e7d50a,2024-04-15 16:28:11,4336197351748828,Cash Withdrawal,2.69,34.961007,-120.410430,Santa Maria,US,0
60084,7bd96a22247346658b23f5e77199157a,2024-04-18 18:28:11,4336197351748828,Cash Withdrawal,16.80,34.968168,-120.418600,Santa Maria,US,0


In [8]:
trans_df.drop(['fraud_label'], inplace = True, axis=1)

In [9]:
trans_df = cc_features.card_owner_age(trans_df, profiles_df)
trans_df = cc_features.expiry_days(trans_df, credit_cards_df)
trans_df = cc_features.activity_level(trans_df, 1)

In [10]:
window_len = 4
window_aggs_df = cc_features.aggregate_activity_by_hour(trans_df, window_len)

Next, you will create features that for each credit card aggregate data from multiple time steps.

Yoy will start by computing the distance between consecutive transactions, lets call it `loc_delta`.
Here you will use the [Haversine distance](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html?highlight=haversine#sklearn.metrics.pairwise.haversine_distances) to quantify the distance between two longitude and latitude coordinates.

Next lets compute windowed aggregates. Here you will use 4-hour windows, but feel free to experiment with different window lengths by setting `window_len` below to a value of your choice.

In [11]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549016
Connected. Call `.close()` to terminate connection gracefully.


To create a feature group you need to give it a name and specify a primary key. It is also good to provide a description of the contents of the feature group and a version number, if it is not defined it will automatically be incremented to `1`.

In [12]:
trans_fg = fs.get_feature_group(name="cc_trans_fraud", version=2)
trans_fg.insert(trans_df, write_options={"wait_for_job" : False})

Uploading Dataframe: 100.00% |██████████| Rows 60086/60086 | Elapsed Time: 00:10 | Remaining Time: 00:00


Launching job: cc_trans_fraud_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549016/jobs/named/cc_trans_fraud_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f3a330d77c0>, None)

In [13]:
window_aggs_fg = fs.get_feature_group(name=f"cc_trans_fraud_{window_len}h", version=2)
window_aggs_fg.insert(window_aggs_df, write_options={"wait_for_job" : False})

Uploading Dataframe: 100.00% |██████████| Rows 60086/60086 | Elapsed Time: 00:08 | Remaining Time: 00:00


Launching job: cc_trans_fraud_4h_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549016/jobs/named/cc_trans_fraud_4h_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f3a330c7b50>, None)

In [14]:

labels_fg = fs.get_feature_group(name="transactions_fraud_label", version=2)
labels_fg.insert(fraud_labels)

Uploading Dataframe: 100.00% |██████████| Rows 60086/60086 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: transactions_fraud_label_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549016/jobs/named/transactions_fraud_label_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f3a3472f8b0>, None)